In [5]:
import sklearn
from efficientnet.keras import preprocess_input
import os
import librosa
import librosa.display
from lime.lime_tabular import LimeTabularExplainer
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from tensorflow.python.keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.callbacks import ModelCheckpoint
from datetime import datetime 
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.python.keras.optimizers import adam_v2
from sklearn import metrics
from tensorflow.python.keras.callbacks import ModelCheckpoint
from datetime import datetime 

In [6]:
trainpath= ("./reduce_sound/")

# get the training classes
keys=os.listdir(trainpath)
values=range(len(keys))

training_classes = np.array(keys)
print('training_classes = ',training_classes)


training_classes =  ['grey-cheeked warbler' 'grey-sided laughingthrush' 'himalayan cuckoo'
 'indian spot-billed duck' 'mountain scops owl' 'mountain tailorbird'
 'pale blue flycatcher' 'scaly laughingthrush' 'grey-winged blackbird']


In [7]:
train_label=[]
folder_item_numbers = []
total_files=0
for folder in  os.listdir(trainpath ) : 
    train_label.append(folder)
    files = glob(pathname= str( trainpath  + folder + '/*.wav'))
    folder_item_numbers.append(len(files))
    total_files= total_files+len(files)
print('--------------------------------------------------')        
foldernames=pd.DataFrame({'Folder_name':train_label})
itemnumbers=pd.DataFrame({'Number of Files':folder_item_numbers})
informations=pd.concat([foldernames,itemnumbers],axis=1)
print(informations)
print('--------------------------------------------------')  
print('Totall Number of Files:                   {}'.format(total_files)) 

--------------------------------------------------
                 Folder_name  Number of Files
0       grey-cheeked warbler              150
1  grey-sided laughingthrush              162
2           himalayan cuckoo              168
3    indian spot-billed duck              156
4         mountain scops owl              120
5        mountain tailorbird              144
6       pale blue flycatcher              168
7       scaly laughingthrush              132
8      grey-winged blackbird              150
--------------------------------------------------
Totall Number of Files:                   1350


In [8]:
def extract(file):
    audio, sample_rate = librosa.load(file) 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    return mfccs_scaled_features

In [9]:
extracted_features=[]
final_df=[]
row=0
for folder in os.listdir(trainpath):
    audio= glob(trainpath+folder+"/*.wav")
    for i in range(len(audio)):
        s=os.path.join(os.path.abspath(audio[i]))
        feature=extract(s)
        print('\nExtracting Feature: {} / {} Files of the classs: {}\nPath: {}'.format(i,len(audio),folder, audio[i]))
        extracted_features.append([feature,folder])
        row_feature = []
        for i in range (len(feature)):
            row_feature.append(feature[i])
        row_feature.append(folder)
        print('\nRow added {} \n '.format(len(final_df)))
        row = row+1
        print(row_feature)
        final_df.append(row_feature)



Extracting Feature: 0 / 150 Files of the classs: grey-cheeked warbler
Path: ./Arunachal_Pradesh/grey-cheeked warbler/phylloscopus poliogenys0.wav

Row added 0 
 
[-472.10077, 58.775963, 7.6819854, 23.957186, 0.5927177, 16.293337, 2.3274417, 7.689467, 4.08543, 5.579339, 0.878765, 7.2429514, 1.0136265, 5.158349, 0.9515723, 4.718149, -1.6044178, 6.2166142, -1.5918177, 5.3788652, -0.50401205, 3.7143116, 0.35114774, 3.4020994, -0.4436758, 4.263005, -1.5043737, 4.3749285, -1.5044032, 3.7154527, -0.42449796, 3.2952769, -1.2969853, 2.8804924, -1.4427377, 2.679581, -0.66103876, 2.3002865, -0.9078947, 2.5063527, 'grey-cheeked warbler']

Extracting Feature: 1 / 150 Files of the classs: grey-cheeked warbler
Path: ./Arunachal_Pradesh/grey-cheeked warbler/phylloscopus poliogenys1.wav

Row added 1 
 
[-207.69487, 77.35819, -61.41406, -14.969862, -31.636604, -9.872795, -25.993723, -1.3372322, -21.952152, -4.8732495, -7.199119, -6.0513983, -5.509097, -4.53123, -1.5218538, -6.18094, -4.116106, 0.962121

In [13]:

extracted_features_df=pd.DataFrame(final_df)
for i in range(40):
    c="mfcc"+str(i)
    extracted_features_df.rename(columns={i:c},inplace=True)
extracted_features_df.to_excel('sample_data.xlsx', sheet_name='sheet1', index=False)
extracted_features_df

,mfcc0,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,...,mfcc31,mfcc32,mfcc33,mfcc34,mfcc35,mfcc36,mfcc37,mfcc38,mfcc39,40
0,-472.100769,58.775963,7.681985,23.957186,0.592718,16.293337,2.327442,7.689467,4.085430,5.579339,...,3.295277,-1.296985,2.880492,-1.442738,2.679581,-0.661039,2.300287,-0.907895,2.506353,grey-cheeked warbler
1,-207.694870,77.358192,-61.414059,-14.969862,-31.636604,-9.872795,-25.993723,-1.337232,-21.952152,-4.873250,...,3.700518,-2.958734,1.013745,-0.629764,0.694357,-0.978971,1.358345,-1.669974,3.239988,grey-cheeked warbler
2,-261.329163,29.816236,-31.988003,1.462996,-49.974438,11.225252,-25.241642,-6.782451,-9.630630,-9.459138,...,1.184512,-1.299144,1.628682,-1.588039,2.328197,-1.929877,2.697136,-1.787442,2.285921,grey-cheeked warbler
3,-384.289886,52.660557,3.222627,43.804008,-44.171955,39.599606,-18.116249,-8.388134,-13.265173,-7.556259,...,0.478428,-0.638380,4.872455,-1.161416,3.526264,4.094943,-0.207011,0.982857,0.585506,grey-cheeked warbler
4,-349.680603,39.620228,-27.916094,71.358826,9.009825,13.436577,7.644772,21.300678,13.501300,12.166832,...,4.008350,-0.149722,2.070220,0.422327,2.408320,-0.277687,2.304784,-0.372489,1.658615,grey-cheeked warbler
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,-479.472260,29.047907,7.356060,63.971088,-15.386018,19.457771,-0.096264,28.393785,3.458495,2.390353,...,4.431319,0.327829,3.458135,1.079991,2.386819,1.115386,2.788863,-0.122470,2.998264,grey-winged blackbird
1346,-427.818329,74.157700,-1.176452,21.496334,0.987784,-2.916337,3.368738,-5.718202,-27.392353,-14.431263,...,-0.723891,-3.960908,-2.399345,-2.466062,-0.935336,-2.952025,-0.932121,-3.251338,-1.288571,grey-winged blackbird
1347,-471.515503,32.497757,-8.082848,18.765202,-0.467873,-0.629512,5.619266,2.159680,-3.812944,1.940123,...,1.074213,0.671645,1.134285,0.292374,0.298281,0.942189,0.645311,0.427531,0.275288,grey-winged blackbird
1348,-489.081024,-5.154406,-75.384186,12.327448,18.345062,-1.690116,4.089919,5.104404,-15.614890,0.782479,...,0.043740,0.411562,1.658569,0.654987,0.547424,0.984870,1.226475,0.354255,0.056419,grey-winged blackbird
